In [1]:
import numpy as np
import tensorflow as tf
import logging
import time
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import cv2
import pandas as pd
from pathlib import Path

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Configure logging
logging.basicConfig(filename='bmi.log', level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

# Load CSV data
df = pd.read_csv("bmi data set.csv")

# Define function to get index of the first digit in a string
def get_index_of_digit(string):
    for i, char in enumerate(string):
        if char.isdigit():
            return i
    return -1

# Function to preprocess and encode full-body images
def encode_full_body_image(image_path):
    # Load and preprocess the image
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = image / 255.0  # Normalize pixel values
    return image

# Load full-body image paths and corresponding IDs
data_folder = "sample_faces"
all_files = list(Path(data_folder).glob("*.[jJpP][pPnN][gG]"))
id_dir = [(Path(images).stem[:get_index_of_digit(Path(images).stem)], images) for images in all_files]

# Create DataFrame from full-body image paths and corresponding IDs
image_df = pd.DataFrame(id_dir, columns=['UID', 'path'])

# Merge full-body image DataFrame with CSV data
data_df = image_df.merge(df, on='UID')

# Prepare data for training
X_train = []
y_height_train = []
y_weight_train = []

# Iterate through each image file
for image_file in data_df['path']:
    uid = Path(image_file).stem.split("_")[0]  # Extract UID from filename
    
    # Encode the full-body image
    encoded_image = encode_full_body_image(image_file)
    X_train.append(encoded_image)
    
    # Extract corresponding height and weight from CSV data
    height = data_df.loc[data_df['path'] == image_file, 'height'].values[0]
    weight = data_df.loc[data_df['path'] == image_file, 'weight'].values[0]
    y_height_train.append(height)
    y_weight_train.append(weight)

# Convert lists to arrays
X_train = np.array(X_train)
y_height_train = np.array(y_height_train)
y_weight_train = np.array(y_weight_train)

# Define the CNN model architecture for height prediction
model_height = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)  # Output layer for height prediction
])

# Compile the height model
optimizer_height = tf.keras.optimizers.Adam(learning_rate=1e-4)  # Adjust learning rate
model_height.compile(optimizer=optimizer_height, loss='mean_squared_error')

# Train the height model
model_height.fit(X_train, y_height_train, epochs=10, batch_size=32)

# Define the CNN model architecture for weight prediction
model_weight = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)  # Output layer for weight prediction
])

# Compile the weight model
optimizer_weight = tf.keras.optimizers.Adam(learning_rate=1e-4)  # Adjust learning rate
model_weight.compile(optimizer=optimizer_weight, loss='mean_squared_error')

# Train the weight model 
model_weight.fit(X_train, y_weight_train, epochs=10, batch_size=32)

# Function to predict height and weight for a given image
def predict_height_weight(input_img, height_model, weight_model):
    logging.info("Predicting height and weight for image %s", input_img)
    start_time = time.time()
    test_array = np.expand_dims(np.array(encode_full_body_image(input_img)), axis=0)
    height = height_model.predict(test_array)[0][0]*3.28084
    weight = weight_model.predict(test_array)[0][0]
    bmi = weight / ((height / 3.28084) ** 2)
    end_time = time.time()
    runtime = end_time - start_time
    logging.info("Predicted height: %f, weight: %f, BMI: %f, runtime: %s", height, weight, bmi, runtime)
    return {'height': height, 'weight': weight, 'bmi': bmi, 'runtime': runtime}

# Create GUI for image input and prediction display
root = tk.Tk()
root.title("BMI Prediction")


def browse_image():
    file_path = filedialog.askopenfilename()
    img = Image.open(file_path)
    img = img.resize((300, 500), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    panel = tk.Label(root, image=img)
    panel.image = img
    panel.grid(row=0, column=0, columnspan=2)
    predict_button["state"] = tk.NORMAL
    global image_path
    image_path = file_path

def predict_bmi():
    prediction = predict_height_weight(image_path, model_height, model_weight)
    prediction_label.config(text=f"Predicted Height: {prediction['height']:.2f} feet\nPredicted Weight: {prediction['weight']:.2f}\nPredicted BMI: {prediction['bmi']:.2f}")

browse_button = tk.Button(root, text="Browse Image", command=browse_image)
browse_button.grid(row=1, column=0)

predict_button = tk.Button(root, text="Predict BMI", command=predict_bmi, state=tk.DISABLED)
predict_button.grid(row=1, column=1)

prediction_label = tk.Label(root, text="")
prediction_label.grid(row=2, column=0, columnspan=2)

root.mainloop()


E:\abc\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"





Epoch 1/10

2/8 [======>.......................] - ETA: 2:17 - loss: 1.7886

AbortedError: Graph execution error:

Detected at node sequential/conv2d_3/Relu defined at (most recent call last):
  File "E:\abc\lib\runpy.py", line 197, in _run_module_as_main

  File "E:\abc\lib\runpy.py", line 87, in _run_code

  File "E:\abc\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "E:\abc\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance

  File "E:\abc\lib\site-packages\ipykernel\kernelapp.py", line 712, in start

  File "E:\abc\lib\site-packages\tornado\platform\asyncio.py", line 199, in start

  File "E:\abc\lib\asyncio\base_events.py", line 601, in run_forever

  File "E:\abc\lib\asyncio\base_events.py", line 1905, in _run_once

  File "E:\abc\lib\asyncio\events.py", line 80, in _run

  File "E:\abc\lib\site-packages\ipykernel\kernelbase.py", line 510, in dispatch_queue

  File "E:\abc\lib\site-packages\ipykernel\kernelbase.py", line 499, in process_one

  File "E:\abc\lib\site-packages\ipykernel\kernelbase.py", li